# Modelagem - TCC

## 01. Importações das bibliotecas

In [1]:
import math
import shap
import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from scipy.stats import ks_2samp
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV

# Função para calculo do KS
def ks(y, p):
    return np.round(ks_2samp(p[y == 1], p[y == 0])[0] * 100, 2)

#import panther as pt
#from panther.utils.memory_reduce import reduce_mem_usage

## 02. Join e limpeza de base

In [2]:
# 2.1. Leitura de base sas
df_cadastrais = pd.read_sas(r'/home/*/.sas7bdat')
df_consultas  = pd.read_sas(r'/home/*/.sas7bdat')
df_negativas  = pd.read_sas(r'/home/*/.sas7bdat')
df_conceito   = pd.read_sas(r'/home/*/.sas7bdat')

# 2.2. Join de tabelas
df_enr = df_cadastrais.merge(df_consultas, left_on = [], right_on=[], how = 'left')
df_enr = df_enr.merge(df_negativas, left_on = [], right_on=[], how = 'left')
df_enr = df_enr.merge(df_conceito, left_on = [], right_on=[], how = 'left')

# 2.3. Converte todos os caracteres das nomes das variáveis minúsculos em maiúsculo
df_enr.columns = df_enr.columns.str.upper()

# 2.4. Regex para remover variáveis de que contenhám...
remov = 'VL|GO|OUA|GBOUT|GBPRI|FICHA|PCTT|CEP'

# 2.5. Regex para remover variáveis de que contenhám...
drop_vars_buraco_valor = df_enr.filter(regex = remov).columns.tolist()
df_full = df_enr.drop(columns = drop_vars_buraco_valor , axis = 1)

# 2.6. Salva base
df_full2.to_csv('/home/*/.csv', index=False)

# 2.7. Print dimensão
print("Base possui {} linhas e {} colunas. ".format(
    df_full2.shape[0], df_full2.shape[1]))

# 2.8. View base
df_full2.head()

## 3. Leitura de dados

In [3]:
# 3.1. Leitura base
df = pd.read_csv(r'/home/*/.csv')
df.head(10)

# 3.2. Cria variável de Safra
df["SAFRA"] = pd.to_datetime(df["DT_T0"], utc=True)
df['SAFRA'] = df['SAFRA'].dt.strftime('%Y%m').astype('str')

# 3.3. Filtra safras necessárias
df = df[df['SAFRA'].isin(['202004','202005','202006','202007','202008','202009','202010','202011','202012','202101','202102','202103','202104','202105','202106'])]

# 3.4. Print dimensão
print("Base possui {} linhas e {} colunas. ".format(
    df.shape[0], df.shape[1]))

# 3.5. Renomeando variável
df = df.rename(columns={"...": "RESPOSTA"})

Base possui 40050 linhas e 629 colunas. 


#### 3.1. Volumetria geral

In [1]:
# 3.1.1 Verificar volume de registros por data (data pura, safra ou trimestre, definir de acordo com volumetria)
df_vol = df.groupby(['SAFRA']).size().reset_index(name='Frequência')
df_vol['Volumetria Safra'] = round(100*(df_vol['Frequência']/sum(df_vol['Frequência'])),2)
df_vol

NameError: name 'df' is not defined

## 04. Amostragem

A base de dados será separada em 3 partes:
- Treino: 70% da base de desenvolvimento
- Teste: 30% da base de desenvolvimento
- Out-Of-Time:

In [5]:
# 4.1. Colunas de referência (essas colunas não fazem sentido analisarmos)
cols_ref = ['NDOC', 'DT_T0', 'RESPOSTA']#, 'SAFRA']
cols_ref

['NDOC', 'DT_T0', 'RESPOSTA']

In [6]:
# 4.2. Colunas que avaliaremos
feats = [c for c in df.columns if c not in cols_ref] # Com a variável safra

feats2 = feats.copy()
feats2.remove('SAFRA') # Sem a variável safra

In [7]:
# 4.3. Amostra Out-Of-Time (OOT)
oot = df[df['SAFRA'].isin(['202104','202105','202106'])]

X_oot = oot[feats]       # Matriz X (features)
y_oot = oot['RESPOSTA']  # Vetor y (target)

print(X_oot.shape)
print(y_oot.shape)

(9024, 626)
(9024,)


In [8]:
# 4.4. Separando aleatoriamente entre o teste e treino
df_desenv = df[~df['SAFRA'].isin(['202104','202105','202106'])]  # Filtro do desenvolvimento

# Split aleatório na proporção de 80/30 (treino/teste)
X_des, X_val, y_des, y_val = train_test_split(
    df_desenv[feats],      # Matriz X do desenvolvimento (features)
    df_desenv['RESPOSTA'], # vetor y do desenvolvimento (target)
    test_size=0.3,         # proporção da amostra de teste
    random_state=2022      # semente aleatória
)

print(' Base features Treino: ', X_des.shape, '\n',
      'Base features Treino: ', y_des.shape, '\n\n')

print(' Base features Teste: ', X_val.shape, '\n',
      'Base features Teste: ', y_val.shape)

 Base features Treino:  (21718, 626) 
 Base features Treino:  (21718,) 


 Base features Teste:  (9308, 626) 
 Base features Teste:  (9308,)


#### Volumetria amostras (DES)

In [9]:
# 4.5. Verificar volume de registros por data (data pura, safra ou trimestre, definir de acordo com volumetria)

DES = X_des.join(y_des)

df_DES = DES.groupby(['SAFRA']).size().reset_index(name='Frequência')
df_DES['Volumetria Safra'] = round(100*(df_DES['Frequência']/sum(df_DES['Frequência'])),2)
df_DES

,SAFRA,Frequência,Volumetria Safra
0,202004,1648,7.59
1,202005,1661,7.65
2,202006,1687,7.77
3,202007,1751,8.06
4,202008,1724,7.94
5,202009,1780,8.20
6,202010,1862,8.57
7,202011,1870,8.61
8,202012,1898,8.74
9,202101,1893,8.72


#### Volumetria amostras (VAL)

In [10]:
# 4.6. Verificar volume de registros por data (data pura, safra ou trimestre, definir de acordo com volumetria)

VAL = X_val.join(y_val)

df_VAL = VAL.groupby(['SAFRA']).size().reset_index(name='Frequência')
df_VAL['Volumetria Safra'] = round(100*(df_VAL['Frequência']/sum(df_VAL['Frequência'])),2)
df_VAL

,SAFRA,Frequência,Volumetria Safra
0,202004,685,7.36
1,202005,707,7.60
2,202006,705,7.57
3,202007,765,8.22
4,202008,793,8.52
5,202009,753,8.09
6,202010,763,8.20
7,202011,780,8.38
8,202012,852,9.15
9,202101,813,8.73


#### Volumetria amostras (OOT)

In [11]:
# 4.7. Verificar volume de registros por data (data pura, safra ou trimestre, definir de acordo com volumetria)

oot2 = df[df['SAFRA'].isin(['202104','202105','202106'])]

df_vol = oot2.groupby(['SAFRA']).size().reset_index(name='Frequência')
df_vol['Volumetria Safra'] = round(100*(df_vol['Frequência']/sum(df_vol['Frequência'])),2)
df_vol

,SAFRA,Frequência,Volumetria Safra
0,202104,2940,32.58
1,202105,3006,33.31
2,202106,3078,34.11


## 05. Univariada

#### 5.1. Verificando variáveis com valores missing (sem preenchimento)

In [12]:
missing_values = pd.DataFrame(df[feats2].isna().sum(), columns = ['QtdMissing']) # Conta quantidade de NA/ausentes
missing_values.sum()
missing_values2 = pd.DataFrame(df[feats2].count(), columns = ['Total'])          # Total da columa preenchida

missing_values_final = pd.merge(missing_values, missing_values2, left_index=True, right_index=True) # Join by index

missing_values_final[missing_values['QtdMissing'] > 0]

,QtdMissing,Total


##### 5.2. Removendo variáveis por concentração (valores iguais)

In [2]:
# 5.2.1. Remoção de variáveis por variabilidade

feats_drop_univ = [] # Iniciando lista vazia para incluir features para serem removidas
tresh_univ = 0.80 # Limite de variabilidade mínima (no máximo 90% de valores iguais)

# 5.2.2. Loop percorrendo todas as features
for f in feats2:
    
    # Percentual máximo de valores iguais
    pct_max = df[f].value_counts(normalize = True).values[0]
    
    # Caso este percentual seja superior ao limite determinado, a feature será removida
    if pct_max > tresh_univ:
        feats_drop_univ.append(f) # Incluindo na lista a feature para ser dropada

# 5.2.3. Print da quantidade de features dropadas
print(f"Drop Univariada: {len(feats_drop_univ)}")

NameError: name 'feats2' is not defined

In [14]:
# 5.2.4. Removendo as features dropadas da lista inicial (apenas as que usaremos para modelar)
feats3 = [f for f in feats2 if f not in feats_drop_univ]

# Quantidade de features disponíveis, após esta análise
print(f"Features disponíveis: {len(feats3)}")

Features disponíveis: 304


#### 5.3. Univariada

In [15]:
def univariada_plus (dataset):
    dataset = dataset.describe(percentiles = [0.99, 0.01, 0.05, 0.95])
    
    dataset = dataset.T
    
    dataset['VARIABILIDADE'] = np.where((dataset['99%'] == 0) | (dataset['99%'] == np.NaN), 'A. Sem variabilidade',
         (np.where(dataset['1%'] == dataset['99%'], 'B. 98% igual',
                   (np.where(dataset['5%'] == dataset['95%'], 'C. 90% igual', 'D. -OK')))))
    
    pct_max = df[f].value_counts(normalize = True).values[0]
    if pct_max > tresh_univ:
        feats_drop_univ.append(f) # Incluindo na lista a feature para ser dropada

        
    return dataset

In [16]:
df2 = univariada_plus(df[feats3]).sort_values("VARIABILIDADE"); df2

,count,mean,std,min,1%,5%,50%,95%,99%,max,VARIABILIDADE
QTSOCIOSPF,40050.0,2.160375,3.903218,-1.0,-1.0,0.0,2.0,5.0,11.0,304.0,D. -OK
QTRESTREXC90D,40050.0,1.336479,6.326801,0.0,0.0,0.0,0.0,6.0,21.0,535.0,D. -OK
QTRESTREXC60D,40050.0,0.840499,4.356814,0.0,0.0,0.0,0.0,4.0,14.0,334.0,D. -OK
QTRESTRRESU24M,40050.0,11.909263,39.728988,0.0,0.0,0.0,3.0,50.0,161.0,1999.0,D. -OK
QTRESTRRESU12M,40050.0,5.941498,21.440015,0.0,0.0,0.0,1.0,25.0,83.0,1311.0,D. -OK
...,...,...,...,...,...,...,...,...,...,...,...
COCONSANUALSEG,40050.0,3.909938,1.583093,0.0,0.0,0.0,5.0,5.0,5.0,5.0,D. -OK
QTCONSORIGSEGUROS720D,40050.0,11.949738,13.115981,0.0,0.0,0.0,8.0,36.0,58.0,198.0,D. -OK
QTCONSORIGSEGUROS360D,40050.0,6.149888,7.729256,0.0,0.0,0.0,4.0,19.0,31.0,117.0,D. -OK
TPMEDCONS5D,40050.0,-0.654152,1.239421,-2.0,-2.0,-2.0,-1.0,2.0,3.0,4.0,D. -OK


In [17]:
univariada_plus(df[feats3])['VARIABILIDADE'].value_counts()

D. -OK    304
Name: VARIABILIDADE, dtype: int64

# 06. Tratamento de valores especiais

In [3]:
# 6.1. Listando os valores especiais da base
def infer_spec_vl(frame: pd.DataFrame, columns: list, spec_vl_limit: float = -1) -> list:
    """Infere valores especiais como todos aqueles menores que o limite."""
    spec_vl = set()
    for col in columns:
        if pd.api.types.is_numeric_dtype(frame[col]):
            series = frame[col]
            uniques = series[series <= spec_vl_limit].unique()
            spec_vl.update(uniques.tolist())
            
    return list(spec_vl)


spec_vl = infer_spec_vl(df, columns=feats, spec_vl_limit=-1)
spec_vl

NameError: name 'pd' is not defined

In [19]:
# 6.2. Tratando valores especiais: one-hot encode + substituição pela mediana
for feature in feats3:
    # Calculando a mediana após filtrar os valores especiais
    median = X_des.loc[X_des[feature] > -1, feature].median()
    
    for vl in [-1]:
        # Criando variáveis dummy (one-hot encode)
        X_des[f'{feature}_{vl}'] = np.where(X_des[feature] == vl, 1, 0)
        X_val[f'{feature}_{vl}'] = np.where(X_val[feature] == vl, 1, 0)
        X_oot[f'{feature}_{vl}'] = np.where(X_oot[feature] == vl, 1, 0)
    
    # Substituindo o valor especial pela mediana na variável original
    X_des[feature].replace({-1: median, -2: median}, inplace = True)
    X_val[feature].replace({-1: median, -2: median}, inplace = True)
    X_oot[feature].replace({-1: median, -2: median}, inplace = True)

## 07. Modelos rápidos e escolha da técnica

Nesta etapa serão testados 2 modelos rápidos de _Machine Learning_ distintos para a escolha da técnica que será utilizada.

#### 7.1. Random Search e modelos sem fit

In [20]:
# 7.1.1. Conjuntos de hiperparâmetros para LGBM
params_rs_lgbm = {
    'num_leaves': list(range(2, 12, 1)),  # Quantidade máxima permitida de nós folhas (filhos) das árvores.
    'learning_rate': list(np.logspace(np.log10(0.005), np.log10(0.2), num = 1000)), # Retornar 1000 números espaçados uniformemente em uma escala logarítmica.
    'min_child_samples': list(range(1100, 2000, 100)),
    'max_depth': list(range(3, 6, 1)),
    'reg_alpha': list(np.logspace(np.log10(0.005), np.log10(5), num = 20)),
    'reg_lambda': list(np.logspace(np.log10(0.005), np.log10(5), num = 20))
    }

# 7.1.2. Conjuntos de hiperparâmetros para LGBM
params_rs_xgb = {
    'num_leaves': list(range(2, 12, 1)),  # Quantidade máxima permitida de nós folhas (filhos) das árvores.
    'learning_rate': list(np.logspace(np.log10(0.005), np.log10(0.2), num = 1000)), # Retornar 1000 números espaçados uniformemente em uma escala logarítmica.
    'min_child_weight': list(range(100, 600, 50)),
    'max_depth': list(range(3, 6, 1)),
    'reg_alpha': list(np.logspace(np.log10(0.005), np.log10(5), num = 20)),
    'reg_lambda': list(np.logspace(np.log10(0.005), np.log10(5), num = 20))
    }

# 7.1.3. Modelo LGBM 
lgb = LGBMClassifier(
    random_state = 2022,  # semente aleatória
    importance_type = 'gain'
)

# 7.1.4. Modelo XGBoost
xgb = XGBClassifier(
    random_state=2022,    # semente aleatória
    importance_type = 'gain'
)

#### 7.2. Random Search - LGBM

In [4]:
# 7.2.1. Função KS para utilizar dentro do random search
# https://stackoverflow.com/questions/65967642/multi-scoring-input-randomizedsearchcv
def ks_scoring(estimator, X, y):
    y_pred = estimator.predict_proba(X)[:,0]
    return ks_2samp( y_pred[y == 0], y_pred[y == 1] )[0]

scoring = {'KS': ks_scoring , 'AUC': 'roc_auc'}

# 7.2.2. Chamada do Random Search
rs_lgbm = RandomizedSearchCV(
    estimator = lgb, # Estimador escolhido
    param_distributions = params_rs_lgbm, # Distribuição dos hiperparâmetros

    scoring = scoring, 
        
    # Função de métrica de performance
    n_jobs = 10, # Quantidade de processamentos em paralelo
    cv = 3, # Quantidades de folds (k-fold)
    n_iter = 12, # Quantidade de iterações (fits) - Número de modelos
    return_train_score = True, # Retorna a métrica (scoring) do treino no cv_results_
    verbose = 1, # Print das etapas
    random_state = 2022, # Semente aleatória,
    refit = 'KS'
).fit(X_des[feats3], y_des)

NameError: name 'RandomizedSearchCV' is not defined

In [22]:
rs_lgbm.best_estimator_

LGBMClassifier(importance_type='gain', learning_rate=0.13773996443434333,
               max_depth=3, min_child_samples=1200, num_leaves=7,
               random_state=2022, reg_alpha=0.044293339520504106,
               reg_lambda=1.1678607345450605)

#### 7.3. Resultados

In [23]:
# # 7.3.1. Tabela de resultados
rs_results = pd.DataFrame(rs_lgbm.cv_results_)[['mean_fit_time', 'params', 'mean_train_KS', 'mean_test_KS', 'rank_test_KS', 
                                                                           'mean_train_AUC', 'mean_test_AUC', 'rank_test_AUC']]

rs_results['estabilidade_KS'] = rs_results['mean_test_KS']/rs_results['mean_train_KS']-1
rs_results['estabilidade_AUC'] = rs_results['mean_test_AUC']/rs_results['mean_train_AUC']-1

rs_results = rs_results.sort_values('mean_test_KS', ascending = False)

rs_results.head(5)

,mean_fit_time,params,mean_train_KS,mean_test_KS,rank_test_KS,mean_train_AUC,mean_test_AUC,rank_test_AUC,estabilidade_KS,estabilidade_AUC
6,25.333147,"{'reg_lambda': 1.1678607345450605, 'reg_alpha'...",0.572864,0.544633,1,0.863289,0.844151,1,-0.049280,-0.022169
5,25.996312,"{'reg_lambda': 1.6799091431418902, 'reg_alpha'...",0.558957,0.544340,2,0.855474,0.842234,5,-0.026149,-0.015477
9,25.359337,"{'reg_lambda': 1.6799091431418902, 'reg_alpha'...",0.553819,0.544146,3,0.851693,0.843166,4,-0.017465,-0.010012
7,25.934731,"{'reg_lambda': 3.4759639808878022, 'reg_alpha'...",0.566766,0.544036,4,0.858062,0.843715,3,-0.040104,-0.016720
10,21.663146,"{'reg_lambda': 0.5644189458423441, 'reg_alpha'...",0.546910,0.542642,5,0.846640,0.841568,7,-0.007803,-0.005990


In [24]:
# # 7.3.2. Filtrar apenas os resultados com menores estabilidades de 10% de queda de KS (modelos mais estáveis)
rs_results_ = rs_results[rs_results['estabilidade_KS'] >= -0.012]\
    .sort_values('rank_test_KS') # Ordenação pelo ks (médio) da validação (teste)

rs_results_.to_csv('/home/cdsw/rs_results_LGBM.csv', index=False)

rs_results_

,mean_fit_time,params,mean_train_KS,mean_test_KS,rank_test_KS,mean_train_AUC,mean_test_AUC,rank_test_AUC,estabilidade_KS,estabilidade_AUC
10,21.663146,"{'reg_lambda': 0.5644189458423441, 'reg_alpha'...",0.546910,0.542642,5,0.846640,0.841568,7,-0.007803,-0.005990
11,15.848270,"{'reg_lambda': 2.416465119285876, 'reg_alpha':...",0.548414,0.542605,6,0.847583,0.841468,8,-0.010592,-0.007216
4,26.373140,"{'reg_lambda': 0.5644189458423441, 'reg_alpha'...",0.537991,0.533942,11,0.840096,0.835975,12,-0.007526,-0.004905
1,15.935089,"{'reg_lambda': 1.6799091431418902, 'reg_alpha'...",0.536491,0.533018,12,0.839957,0.836123,11,-0.006473,-0.004565


In [25]:
best_ranks = np.sort(rs_results_['rank_test_KS'].values) # Rank dos melhores modelos
best_ranks.shape

(4,)

In [26]:
rs_results_['params'][1]

{'reg_lambda': 1.6799091431418902,
 'reg_alpha': 0.06371374928515666,
 'num_leaves': 4,
 'min_child_samples': 1700,
 'max_depth': 4,
 'learning_rate': 0.015026636248522325}

#### 7.4. Seleção do melhor modelo - lgbm

Ajustaremos os modelos novamente na base de treino e testaremos na amostra de teste e de OOT, visando calcular a performance real em amostras nunca vistas pelo estimador. Consolidaremos os resultados a cada iteração e selecionaremos o melhor modelo em relação a performance e estabilidade nas bases de validações.

In [27]:
# 7.4.1. Loop para calcular métricas dos melhores modelos

best_results = pd.DataFrame() # DataFrame vazio que será preenchido com informações do processo

# Loop pelos ranks dos melhores modelos
for r in best_ranks:
    
    tic = datetime.now() # Início do processamento
    print(f'[Modelo: {str(r).zfill(3)}]:          ', end = '')
    
    mask = rs_results_['rank_test_KS'] == r # objeto com o filtro do rank específico
    params_ = rs_results_.loc[mask, 'params'].values[0] # selecionando os hiperparâmetros de acordo com o filtro

    # LGBM
    lgb = LGBMClassifier(
        **params_, # atribuindo todos os hiperparâmetros (dicionário)
        random_state = 2022, # semente aleatória
        n_jobs = 10, # quantidade de processadores
        importance_type = 'gain' # tipo de importancia
    ).fit(X_des[feats3], y_des)

    toc = datetime.now() # Final do processamento do modelo
    lgb_time = toc - tic # tempo de processamento do modelo
    
    p_train_lgb = lgb.predict_proba(X_des[feats3])[:, 1] # probabilidade do treino
    p_test_lgb = lgb.predict_proba(X_val[feats3])[:, 1] # probabilidade do teste
    p_oot_lgb = lgb.predict_proba(X_oot[feats3])[:, 1] # probabilidade da OOT
    
    ks_train = ks(y_des, p_train_lgb) # KS treino
    ks_test = ks(y_val, p_test_lgb) # KS teste
    ks_oot = ks(y_oot, p_oot_lgb) # KS OOT
    
    # ROC
    roc_train = np.round(100*roc_auc_score(y_des, lgb.predict_proba(X_des[feats3])[:, 1]),2)
    roc_test = np.round(100*roc_auc_score(y_val, lgb.predict_proba(X_val[feats3])[:, 1]),2)
    roc_oot = np.round(100*roc_auc_score(y_oot, lgb.predict_proba(X_oot[feats3])[:, 1]),2)
        
    # Juntando informações do processamento
    best_results = pd.concat([
        best_results,
        pd.DataFrame({'rank': [r], 'time_process': [lgb_time], 'params': [params_], 'ks_train': [ks_train], 'ks_test': [ks_test], 'ks_oot': [ks_oot],
                                                                                    'roc_train': [roc_train], 'roc_test': [roc_test], 'roc_oot': [roc_oot]})
    ], axis = 0).reset_index(drop=True)
    
#    print(f'KS TRAIN: {ks_train}  |  KS TEST: {ks_test}  |  KS OOT: {ks_oot}')
    print(f'KS TRAIN: {ks_train}  | KS TEST: {ks_test}  | KS OOT: {ks_oot}  | ROC TRAIN: {roc_train}  | ROC OOS: {roc_test}  | ROC OOT: {roc_oot}')

# 7.4.2. Resetando o index
best_results.reset_index(inplace = True)

# 7.4.3. Criando métricas de estabilidade
best_results['estab_test'] = best_results['ks_test']/best_results['ks_train']-1
best_results['estab_oot'] = best_results['ks_oot']/best_results['ks_train']-1

# 7.4.4. Salvando resultados
best_results.to_pickle('/home/*/ascend_best_results_lgbm.pkl')
best_results.to_csv('/home/*/Fast_KS_AMOSTRAS_lgb.csv')

# Escolheremos o modelo 11 QUE é o 4 no rs_results_

[Modelo: 005]:          KS TRAIN: 54.58  | KS TEST: 56.0  | KS OOT: 53.68  | ROC TRAIN: 84.56  | ROC OOS: 84.98  | ROC OOT: 83.64
[Modelo: 006]:          KS TRAIN: 54.84  | KS TEST: 55.78  | KS OOT: 53.53  | ROC TRAIN: 84.72  | ROC OOS: 85.03  | ROC OOT: 83.75
[Modelo: 011]:          KS TRAIN: 53.71  | KS TEST: 55.37  | KS OOT: 53.4  | ROC TRAIN: 83.99  | ROC OOS: 84.52  | ROC OOT: 83.2
[Modelo: 012]:          KS TRAIN: 53.55  | KS TEST: 55.28  | KS OOT: 53.16  | ROC TRAIN: 83.91  | ROC OOS: 84.55  | ROC OOT: 83.21


#### 7.5. Verificando quantas variáveis importantes o modelo escolhido tem

In [28]:
# 7.5.1. LGBM
lgb = LGBMClassifier(**rs_results_['params'][4],
random_state = 2022, # semente aleatória
n_jobs = 10, # quantidade de processadores
importance_type = 'gain'                     
) # Valor da importância das features será o ganho delas nas árvores

lgb.fit(X_des[feats3], y_des) # Fit do modelo com o X e y de treino

# 7.5.2. Aplicando o modelo (probabilidade) nas amostras. (Lembrar que mau é 1)
p_train_lgb = lgb.predict_proba(X_des[feats3])[:, 1]  # probabilidade do treino
p_test_lgb = lgb.predict_proba(X_val[feats3])[:, 1]    # probabilidade do teste
p_oot_lgb = lgb.predict_proba(X_oot[feats3])[:, 1]      # probabilidade da OOT

# 7.5.3. Print do KS em cada uma das amostras
print('--> LightGBM')
print(f'KS TRAIN: {ks(y_des, p_train_lgb)}')
print(f'KS TEST:  {ks(y_val, p_test_lgb)}')
print(f'KS OOT:   {ks(y_oot, p_oot_lgb)}')

# 7.5.4. Criar um DataFrame com as importâncias referenciando as features
df_imp = pd.DataFrame({
  'feature': feats3,
  'importance': lgb.feature_importances_
})

# 7.5.5. Features com importância zero, vamos remove-las
feats_drop_mfast = df_imp.loc[df_imp['importance'] == 0, 'feature'].tolist()

# 7.5.6. Quantidade de features que serão dropadas por importância zerada
print(f"Drop Importância Zero: {len(feats_drop_mfast)}")

# 7.5.7. Ficar com todas as features iniciais, exceto as com importâncias zeradas
feats4 = [f for f in feats3 if f not in feats_drop_mfast]

# 7.5.8. Quantidade de features disponíveis após essa análise
print(f"Features disponíveis: {len(feats4)}")

--> LightGBM
KS TRAIN: 53.71
KS TEST:  55.37
KS OOT:   53.4
Drop Importância Zero: 274
Features disponíveis: 30


#### 7.6. Random Search - XGboost

In [29]:
# 7.6.1. Função KS para utilizar dentro do random search
def ks_scoring(estimator, X, y):
    y_pred = estimator.predict_proba(X)[:,0]
    return ks_2samp( y_pred[y == 0], y_pred[y == 1] )[0]

scoring = {'KS': ks_scoring , 'AUC': 'roc_auc'}

# 7.6.2. Chamada do Random Search
rs_xgb = RandomizedSearchCV(
    estimator = xgb, # Estimador escolhido
    param_distributions = params_rs_xgb, # Distribuição dos hiperparâmetros
    scoring = scoring, # Função de métrica de performance
    n_jobs = 10, # Quantidade de processamentos em paralelo
    cv = 3, # Quantidades de folds (k-fold)
    n_iter = 12, # Quantidade de iterações (fits)
    return_train_score = True, # Retorna a métrica (scoring) do treino no cv_results_
    verbose = 1, # Print das etapas
    random_state = 2022, # Semente aleatória
    refit = 'KS'
).fit(X_des[feats3], y_des)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  36 out of  36 | elapsed:  8.7min finished


#### 7.7. Resultados

In [5]:
# 7.7.1. Tabela de resultados
rs_results = pd.DataFrame(rs_xgb.cv_results_)[['mean_fit_time', 'params', 'mean_train_KS', 'mean_test_KS', 'rank_test_KS', 
                                                                           'mean_train_AUC', 'mean_test_AUC', 'rank_test_AUC']]

rs_results['estabilidade_KS'] = rs_results['mean_test_KS']/rs_results['mean_train_KS']-1
rs_results['estabilidade_AUC'] = rs_results['mean_test_AUC']/rs_results['mean_train_AUC']-1

rs_results = rs_results.sort_values('mean_test_KS', ascending = False)

rs_results.head(5)

NameError: name 'pd' is not defined

In [31]:
# 7.7.2. Filtrar apenas os resultados com menores estabilidades de 10% de queda de KS (modelos mais estáveis)
rs_results_ = rs_results[rs_results['estabilidade_KS'] >= -0.0215]\
    .sort_values('rank_test_KS') # Ordenação pelo ks (médio) da validação (teste)

rs_results_.to_csv('/home/cdsw/rs_results_XGB.csv', index=False)

rs_results_

,mean_fit_time,params,mean_train_KS,mean_test_KS,rank_test_KS,mean_train_AUC,mean_test_AUC,rank_test_AUC,estabilidade_KS,estabilidade_AUC
10,110.048843,"{'reg_lambda': 0.5644189458423441, 'reg_alpha'...",0.552752,0.543137,5,0.849749,0.840792,9,-0.017394,-0.010542
6,105.137339,"{'reg_lambda': 1.1678607345450605, 'reg_alpha'...",0.554351,0.542490,6,0.851390,0.840950,8,-0.021396,-0.012263
0,61.964657,"{'reg_lambda': 0.007192249441438314, 'reg_alph...",0.535919,0.534957,11,0.840118,0.836541,11,-0.001795,-0.004257
4,108.493721,"{'reg_lambda': 0.5644189458423441, 'reg_alpha'...",0.537429,0.531519,12,0.841477,0.836267,12,-0.010997,-0.006192


In [32]:
best_ranks = np.sort(rs_results_['rank_test_KS'].values) # Rank dos melhores modelos
best_ranks.shape

(4,)

In [33]:
rs_results_['params'][0]

{'reg_lambda': 0.007192249441438314,
 'reg_alpha': 0.06371374928515666,
 'num_leaves': 9,
 'min_child_weight': 500,
 'max_depth': 5,
 'learning_rate': 0.01716301828221557}

#### 7.8. Seleção do melhor modelo - XGB

Ajustaremos os modelos novamente na base de treino e testaremos na amostra de teste e de OOT, visando calcular a performance real em amostras nunca vistas pelo estimador. Consolidaremos os resultados a cada iteração e selecionaremos o melhor modelo em relação a performance e estabilidade nas bases de validações.

In [34]:
# 7.8.1. Loop para calcular métricas dos melhores modelos

best_results = pd.DataFrame() # DataFrame vazio que será preenchido com informações do processo

# 7.8.2. Loop pelos ranks dos melhores modelos
for r in best_ranks:
    
    tic = datetime.now() # Início do processamento
    print(f'[Modelo: {str(r).zfill(3)}]:          ', end = '')
    
    mask = rs_results_['rank_test_KS'] == r # objeto com o filtro do rank específico
    params_ = rs_results_.loc[mask, 'params'].values[0] # selecionando os hiperparâmetros de acordo com o filtro

    # XGB
    xgb = XGBClassifier(
        **params_, # atribuindo todos os hiperparâmetros (dicionário)
        random_state = 2022, # semente aleatória
        n_jobs = 10, # quantidade de processadores
        importance_type = 'gain' # tipo de importancia
    ).fit(X_des[feats3], y_des)

    toc = datetime.now() # Final do processamento do modelo
    xgb_time = toc - tic # tempo de processamento do modelo
    
    p_train_xgb = xgb.predict_proba(X_des[feats3])[:, 1] # probabilidade do treino
    p_test_xgb = xgb.predict_proba(X_val[feats3])[:, 1] # probabilidade do teste
    p_oot_xgb = xgb.predict_proba(X_oot[feats3])[:, 1] # probabilidade da OOT
    
    ks_train = ks(y_des, p_train_xgb) # KS treino
    ks_test = ks(y_val, p_test_xgb) # KS teste
    ks_oot = ks(y_oot, p_oot_xgb) # KS OOT
    
    # ROC
    roc_train = np.round(100*roc_auc_score(y_des, xgb.predict_proba(X_des[feats3])[:, 1]),2)
    roc_test = np.round(100*roc_auc_score(y_val, xgb.predict_proba(X_val[feats3])[:, 1]),2)
    roc_oot = np.round(100*roc_auc_score(y_oot, xgb.predict_proba(X_oot[feats3])[:, 1]),2)
    
    # Juntando informações do processamento
    best_results = pd.concat([
        best_results,
        pd.DataFrame({'rank': [r], 'time_process': [xgb_time], 'params': [params_], 'ks_train': [ks_train], 'ks_test': [ks_test], 'ks_oot': [ks_oot],
                                                                                    'roc_train': [roc_train], 'roc_test': [roc_test], 'roc_oot': [roc_oot]})
    ], axis = 0).reset_index(drop=True)
    
    #print(f'KS TRAIN: {ks_train}  |  KS TEST: {ks_test}  |  KS OOT: {ks_oot}')
    print(f'KS TRAIN: {ks_train}  | KS TEST: {ks_test}  | KS OOT: {ks_oot}  | ROC TRAIN: {roc_train}  | ROC OOS: {roc_test}  | ROC OOT: {roc_oot}')

# 7.8.3. Resetando o index
best_results.reset_index(inplace = True)

# 7.8.4. Criando métricas de estabilidade
best_results['estab_test'] = best_results['ks_test']/best_results['ks_train']-1
best_results['estab_oot'] = best_results['ks_oot']/best_results['ks_train']-1

# 7.8.5. Salvando resultados
best_results.to_pickle('/home/cdsw/ascend_best_results_xgb.pkl')
best_results.to_csv('/home/cdsw/Fast_KS_AMOSTRAS_xgb.csv')

# Escolheremos o modelo 11 QUE é o 0 no rs_results_

[Modelo: 005]:          KS TRAIN: 55.75  | KS TEST: 55.95  | KS OOT: 53.44  | ROC TRAIN: 85.43  | ROC OOS: 85.07  | ROC OOT: 83.96
[Modelo: 006]:          KS TRAIN: 55.73  | KS TEST: 56.02  | KS OOT: 53.51  | ROC TRAIN: 85.29  | ROC OOS: 85.13  | ROC OOT: 83.92
[Modelo: 011]:          KS TRAIN: 53.95  | KS TEST: 55.45  | KS OOT: 53.34  | ROC TRAIN: 84.35  | ROC OOS: 84.76  | ROC OOT: 83.43
[Modelo: 012]:          KS TRAIN: 53.84  | KS TEST: 55.32  | KS OOT: 53.12  | ROC TRAIN: 84.06  | ROC OOS: 84.56  | ROC OOT: 83.23


In [35]:
#### Atenção! Refinaremos o modelo de LGBM, pois apresentou, minunciosamente, os melhores resultados.

#### 7.9. Verificando quantas variáveis importantes o modelo escolhido tem

In [36]:
# 7.9.1. LGBM
lgb = XGBClassifier(**rs_results_['params'][0],
random_state = 2022, # semente aleatória
n_jobs = 10, # quantidade de processadores
importance_type = 'gain'                     
) # Valor da importância das features será o ganho delas nas árvores

lgb.fit(X_des[feats3], y_des) # Fit do modelo com o X e y de treino

# 7.9.2. Aplicando o modelo (probabilidade) nas amostras. (Lembrar que mau é 1)
p_train_lgb = lgb.predict_proba(X_des[feats3])[:, 1]  # probabilidade do treino
p_test_lgb = lgb.predict_proba(X_val[feats3])[:, 1]    # probabilidade do teste
p_oot_lgb = lgb.predict_proba(X_oot[feats3])[:, 1]      # probabilidade da OOT

# 7.9.3. Print do KS em cada uma das amostras
print('--> LightGBM')
print(f'KS TRAIN: {ks(y_des, p_train_lgb)}')
print(f'KS TEST:  {ks(y_val, p_test_lgb)}')
print(f'KS OOT:   {ks(y_oot, p_oot_lgb)}')

# 7.9.4. Criar um DataFrame com as importâncias referenciando as features
df_imp = pd.DataFrame({
  'feature': feats3,
  'importance': lgb.feature_importances_
})

# 7.9.5. Features com importância zero, vamos remove-las
feats_drop_mfast = df_imp.loc[df_imp['importance'] == 0, 'feature'].tolist()

# 7.9.6. Quantidade de features que serão dropadas por importância zerada
print(f"Drop Importância Zero: {len(feats_drop_mfast)}")

# 7.9.7. Ficar com todas as features iniciais, exceto as com importâncias zeradas
feats4 = [f for f in feats3 if f not in feats_drop_mfast]

# 7.9.8. Quantidade de features disponíveis após essa análise
print(f"Features disponíveis: {len(feats4)}")

--> LightGBM
KS TRAIN: 53.95
KS TEST:  55.45
KS OOT:   53.34
Drop Importância Zero: 249
Features disponíveis: 55


# 08. Nova rodada de modelos apenas com LGBM aumentando o pull de hyperparametros pra ver o melhor modelo.

In [37]:
# 8.1. Conjuntos de hiperparâmetros
params_rs_lgbm = {
    'num_leaves': list(range(3, 12, 2)),  # Quantidade máxima permitida de nós folhas (filhos) das árvores.
    'learning_rate': list(np.logspace(np.log10(0.005), np.log10(0.3), num = 1000)), # Retornar 1000 números espaçados uniformemente em uma escala logarítmica.
    'min_child_samples': list(range(1100, 2000, 100)),
    'max_depth': list(range(3, 6, 1)),
    'n_estimators': list(range(100, 800, 100)),
    'min_child_weight': list(np.logspace(np.log10(70), np.log10(650), num = 1000)),
    'subsample': list(np.logspace(np.log10(0.5), np.log10(1), num = 20)),
    'colsample_bytree': list(np.logspace(np.log10(0.5), np.log10(1), num = 20)),
    'reg_alpha': list(np.logspace(np.log10(0.005), np.log10(8), num = 100)),
    'reg_lambda': list(np.logspace(np.log10(0.005), np.log10(8), num = 100)),
    'min_split_gain': list(range(0, 20, 2)),
    'gamma': list(range(1, 7, 1)),
}

        
# 8.2. LGBM
lgb = LGBMClassifier(
    random_state = 2022,  # semente aleatória
    importance_type = 'gain'
)

In [ ]:
# 8.3. Função KS para utilizar dentro do random search
def ks_scoring(estimator, X, y):
    y_pred = estimator.predict_proba(X)[:,0]
    return ks_2samp( y_pred[y == 0], y_pred[y == 1] )[0]

scoring = {'KS': ks_scoring , 'AUC': 'roc_auc'}

# 8.4. Chamada do Random Search
rs_lgbm = RandomizedSearchCV(
    estimator = lgb, # Estimador escolhido
    param_distributions = params_rs_lgbm, # Distribuição dos hiperparâmetros
    scoring = scoring, # Função de métrica de performance
    n_jobs = 10, # Quantidade de processamentos em paralelo
    cv = 3, # Quantidades de folds (k-fold)
    n_iter = 500, # Quantidade de iterações (fits) - Número de modelos
    return_train_score = True, # Retorna a métrica (scoring) do treino no cv_results_
    verbose = 1, # Print das etapas
    random_state = 2022, # Semente aleatória
    refit = 'KS'
).fit(X_des[feats3], y_des)

Fitting 3 folds for each of 500 candidates, totalling 1500 fits


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:  3.2min
[Parallel(n_jobs=10)]: Done 180 tasks      | elapsed: 19.1min
[Parallel(n_jobs=10)]: Done 430 tasks      | elapsed: 41.2min


In [ ]:
# 8.5. Tabela de resultados
rs_results = pd.DataFrame(rs_lgbm.cv_results_)[['mean_fit_time', 'params', 'mean_train_KS', 'mean_test_KS', 'rank_test_KS', 
                                                                           'mean_train_AUC', 'mean_test_AUC', 'rank_test_AUC']]

rs_results['estabilidade_KS'] = rs_results['mean_test_KS']/rs_results['mean_train_KS']-1
rs_results['estabilidade_AUC'] = rs_results['mean_test_AUC']/rs_results['mean_train_AUC']-1

rs_results = rs_results.sort_values('mean_test_KS', ascending = False)

rs_results.head(5)

In [ ]:
rs_results.to_csv('/home/cdsw/REFINO.csv')

In [ ]:
# 8.6. Filtrar apenas os resultados com menores estabilidades de 10% de queda de KS (modelos mais estáveis)
rs_results_ = rs_results[rs_results['estabilidade_KS'] >= -0.012]\
    .sort_values('rank_test_KS') # Ordenação pelo ks (médio) da validação (teste)

rs_results_

In [ ]:
best_ranks = np.sort(rs_results_['rank_test_KS'].values) # Rank dos melhores modelos
best_ranks.shape

In [ ]:
# 8.7. Loop para calcular métricas dos melhores modelos

best_results = pd.DataFrame() # DataFrame vazio que será preenchido com informações do processo

# 8.8. Loop pelos ranks dos melhores modelos
for r in best_ranks:
    
    tic = datetime.now() # Início do processamento
    print(f'[Modelo: {str(r).zfill(3)}]:          ', end = '')
    
    mask = rs_results_['rank_test_KS'] == r # objeto com o filtro do rank específico
    params_ = rs_results_.loc[mask, 'params'].values[0] # selecionando os hiperparâmetros de acordo com o filtro

    # LGBM
    lgb = LGBMClassifier(
        **params_, # atribuindo todos os hiperparâmetros (dicionário)
        random_state = 2022, # semente aleatória
        n_jobs = 10,
        importance_type = 'gain'
        # quantidade de processadores
    ).fit(X_des[feats3], y_des)

    toc = datetime.now() # Final do processamento do modelo
    lgb_time = toc - tic # tempo de processamento do modelo
    
    p_train_lgb = lgb.predict_proba(X_des[feats3])[:, 1] # probabilidade do treino
    p_test_lgb = lgb.predict_proba(X_val[feats3])[:, 1] # probabilidade do teste
    p_oot_lgb = lgb.predict_proba(X_oot[feats3])[:, 1] # probabilidade da OOT
    
    ks_train = ks(y_des, p_train_lgb) # KS treino
    ks_test = ks(y_val, p_test_lgb) # KS teste
    ks_oot = ks(y_oot, p_oot_lgb) # KS OOT
    
    # ROC
    roc_train = np.round(100*roc_auc_score(y_des, lgb.predict_proba(X_des[feats3])[:, 1]),2)
    roc_test = np.round(100*roc_auc_score(y_val, lgb.predict_proba(X_val[feats3])[:, 1]),2)
    roc_oot = np.round(100*roc_auc_score(y_oot, lgb.predict_proba(X_oot[feats3])[:, 1]),2)
    
    # Juntando informações do processamento
    best_results = pd.concat([
        best_results,
        pd.DataFrame({'rank': [r], 'time_process': [lgb_time], 'params': [params_], 'ks_train': [ks_train], 'ks_test': [ks_test], 'ks_oot': [ks_oot],
                                                                                    'roc_train': [roc_train], 'roc_test': [roc_test], 'roc_oot': [roc_oot]})
    ], axis = 0).reset_index(drop=True)
    
#    print(f'KS TRAIN: {ks_train}  |  KS TEST: {ks_test}  |  KS OOT: {ks_oot}')
    print(f'KS TRAIN: {ks_train}  | KS TEST: {ks_test}  | KS OOT: {ks_oot}  | ROC TRAIN: {roc_train}  | ROC OOS: {roc_test}  | ROC OOT: {roc_oot}')

# 8.8. Resetando o index
best_results.reset_index(inplace = True)

# 8.9. Criando métricas de estabilidade
best_results['estab_test'] = best_results['ks_test']/best_results['ks_train']-1
best_results['estab_oot'] = best_results['ks_oot']/best_results['ks_train']-1

# 8.10. Salvando resultados
best_results.to_pickle('/home/*/ascend_best_results_refinao.pkl')
best_results.to_csv('/home/*/ascend_refinao.csv')

# 09. Remoção por Importância

#### lendo modelo

In [ ]:
best_results2 = pd.read_csv('/home/cdsw/ascend_refinao.csv');
best_results2
# Baseado nessa tabela escolhar o melhor algoritmo e faça os testes abaixo pra ver.

In [ ]:
best_results2['params'][184]

In [ ]:
# Aplicando o modelo LGBM selecionado
#lgb = LGBMClassifier(**best_results['params'][40],
lgb = LGBMClassifier(subsample = 0.7746298211333279, reg_lambda = 0.04676440937077718, reg_alpha = 5.937868749792548, num_leaves = 3, n_estimators = 100, min_split_gain = 16, min_child_weight = 378.8502727624012, min_child_samples = 1900, max_depth = 4, learning_rate = 0.012167884278142268, gamma = 4, colsample_bytree = 0.8332620063985445,
random_state = 2022, # semente aleatória
n_jobs = 10, # quantidade de processadores
importance_type = 'gain'                     
) # Valor da importância das features será o ganho delas nas árvores

lgb.fit(X_des[feats3], y_des) # Fit do modelo com o X e y de treino

# Aplicando o modelo (probabilidade) nas amostras. (Lembrar que mau é 1)
p_train_lgb = lgb.predict_proba(X_des[feats3])[:, 1]  # probabilidade do treino
p_test_lgb = lgb.predict_proba(X_val[feats3])[:, 1]    # probabilidade do teste
p_oot_lgb = lgb.predict_proba(X_oot[feats3])[:, 1]      # probabilidade da OOT

# Print do KS em cada uma das amostras
print('--> LightGBM')
print(f'KS TRAIN: {ks(y_des, p_train_lgb)}')
print(f'KS TEST:  {ks(y_val, p_test_lgb)}')
print(f'KS OOT:   {ks(y_oot, p_oot_lgb)}')

### 9.1. Importância zerada

#### 9.2. Importância das variáveis

In [ ]:
# Criar um DataFrame com as importâncias referenciando as features

df_imp = pd.DataFrame({
  'feature': feats3,
  'importance': lgb.feature_importances_
})

# Ordenando da mais importante para a menos
df_imp.sort_values('importance', ascending = False, inplace=True)

# Print 5 features mais importantes
df_imp

#### 9.3. Freq. Importância

In [ ]:
# Frequência da importância 
df_imp['importance'].value_counts()

#### 9.4. Remover variáveis zeradas

In [ ]:
# 9.4.1. Features com importância zero, vamos remove-las
feats_drop_mfast = df_imp.loc[df_imp['importance'] == 0, 'feature'].tolist()

# 9.4.2. Quantidade de features que serão dropadas por importância zerada
print(f"Drop Importância Zero: {len(feats_drop_mfast)}")

# 9.4.3. Ficar com todas as features iniciais, exceto as com importâncias zeradas
feats4 = [f for f in feats3 if f not in feats_drop_mfast]

# 9.4.4. Quantidade de features disponíveis após essa análise
print(f"Features disponíveis: {len(feats4)}")

#### 9.5. Verificando modelo pós remoção de variáveis com importância zerada

In [ ]:
# 9.5.1. LGBM
lgb = LGBMClassifier(subsample = 0.7746298211333279, reg_lambda = 0.04676440937077718, reg_alpha = 5.937868749792548, num_leaves = 3, n_estimators = 100, min_split_gain = 16, min_child_weight = 378.8502727624012, min_child_samples = 1900, max_depth = 4, learning_rate = 0.012167884278142268, gamma = 4, colsample_bytree = 0.8332620063985445,
random_state = 2022, # semente aleatória
n_jobs = 10, # quantidade de processadores
importance_type = 'gain'
) # Valor da importância das features será o ganho delas nas árvores

lgb.fit(X_des[feats4], y_des) # Fit do modelo com o X e y de treino

# 9.5.2. Aplicando o modelo (probabilidade) nas amostras. (Lembrar que mau é 1)
p_train_lgb = lgb.predict_proba(X_des[feats4])[:, 1]  # probabilidade do treino
p_test_lgb = lgb.predict_proba(X_val[feats4])[:, 1]    # probabilidade do teste
p_oot_lgb = lgb.predict_proba(X_oot[feats4])[:, 1]      # probabilidade da OOT

# 9.5.3. Print do KS em cada uma das amostras
print('--> LightGBM')
print(f'KS TRAIN: {ks(y_des, p_train_lgb)}')
print(f'KS TEST:  {ks(y_val, p_test_lgb)}')
print(f'KS OOT:   {ks(y_oot, p_oot_lgb)}')

#### 9.6. Importancia

In [ ]:
# 9.6.1. Criar um DataFrame com as importâncias referenciando as features
df_imp = pd.DataFrame({
  'feature': feats4,
  'importance': lgb.feature_importances_
})

# 9.6.2. Ordenando da mais importante para a menos
df_imp.sort_values('importance', ascending = False, inplace=True)

# 9.6.3. Print 5 features mais importantes
df_imp['Perc'] = df_imp['importance']/sum(df_imp['importance'])
df_imp

# 10. Remoção de variável recursirvamente

In [ ]:
# 10.1.1 Função para retirada de vars. recursiva
def recursive_selection(
    features: list,
    X_train: pd.DataFrame,
    X_test: pd.DataFrame,
    y_train: pd.Series,
    y_test: pd.Series,
    step: int = 5,
    max_ks_diff: int = 2,
    n_feats: int = 50
):
    """
    Seleção sequencial de variáveis pelo feature importance, observando o KS a cada iteração.
    """
    feats_ = features.copy() # Cópia da lista de features (caso dê algum problema, temos a lista original)
    i = 0 # indicador de contagem (iniciando em zero)
    summary_rfe = pd.DataFrame() # DataFrame que será inputado os resultados sumarizado do processo
    feat_drop = [] # Lista de features para drop, inicialmente vazia
    print(f'Número de variáveis inicial: {len(feats_)}')

    # Loop infinito (até algum critério de parada ser atendido)
    while True:

        tic = datetime.now() # início do processamento

        # Prints iniciais
        print(f'Iteração {i+1}:') # Iteração
        print(f'  -> {len(feat_drop)} features removidas: {feat_drop}') # Features que serão dropadas
        print(f'  -> {len(feats_)} features disponíveis') # Features disponíveis

        # LGBM
        #lgb = LGBMClassifier(**best_results['params'][40],
        lgb = LGBMClassifier(subsample = 0.7746298211333279, reg_lambda = 0.04676440937077718, reg_alpha = 5.937868749792548, num_leaves = 3, n_estimators = 100, min_split_gain = 16, min_child_weight = 378.8502727624012, min_child_samples = 1900, max_depth = 4, learning_rate = 0.012167884278142268, gamma = 4, colsample_bytree = 0.8332620063985445,

                random_state = 2022, # semente aleatória
                n_jobs = 10, # quantidade de processadores
                importance_type = 'gain').fit(X_train[feats_], y_train) # Fit do modelo com o X e y de treino

        toc = datetime.now() # final do processamento
        lgb_time = toc - tic # tempo total de processamento
        print(f'Tempo de processamento do modelo: {lgb_time}')

        p_train_lgb = lgb.predict_proba(X_train[feats_])[:, 1] # probabilidade do treino
        p_test_lgb = lgb.predict_proba(X_test[feats_])[:, 1] # probabilidade do teste
        p_oot_lgb = lgb.predict_proba(X_oot[feats_])[:, 1] # probabilidade da OOT

        # KS da Out-of-sample (test)
        ks_test = ks(y_test, p_test_lgb)
        print(f'  -> KS test: {ks_test}')

        # KS de referência (caso seja a primeira iteração - modelo inicial)
        if i == 0:
            ks_ref = ks_test
            print(f'  -> KS de referência: {ks_ref}')

        # Concatenação (union) do dataframe de resumo do RFE
        summary_rfe = pd.concat([
            summary_rfe, 
            pd.DataFrame({
                'iteration': [i], # iteração
                'feature_drop': [feat_drop], # lista das features que serão removidas
                'feature': [feats_], # lista das features disponíveis
                'importance': [lgb.feature_importances_.tolist()], # importancia das features disponíveis
                'ks_train': [ks(y_train, p_train_lgb)], # KS do treino
                'ks_test':[ ks(y_test, p_test_lgb)], # KS do teste
                'ks_oot': [ ks(y_oot, p_oot_lgb)], # KS da OOT
                'time_process': [lgb_time] # tempo de processamento do modelo
            })
        ], axis = 0) # append de linhas

        # DataFrame com as importâncias referenciando as features
        df_imp = pd.DataFrame({'feature': feats_, 'importance': lgb.feature_importances_})\
            .sort_values('importance', ascending = False) # Ordenando da mais importante para a menos

        # Caso tenha alguma feature com importância zerada, será considerada como a de menor importância
        if np.sum(df_imp['importance'] == 0) > 0:
            feat_drop = df_imp.loc[df_imp['importance'] == 0, 'feature'].values # lista de features com importância zerada
        else:
            feat_drop = df_imp['feature'].values[-step:] # lista das 'step' features com menor importância

        # Atualizando a lista de features disponíveis -> removendo as features de drop
        feats_ = [f for f in feats_ if f not in feat_drop]

        # Critério de parada da diferença de KS
        if ks_ref - ks_test >= max_ks_diff:
            print(f'Fim do processamento -> Diferença de KS maior que {max_ks_diff} pontos')
            break

        # Critério de parada da quantidade de features disponíveis finais
        if len(feats_) < n_feats:
            print('Fim do processamento -> Número de features atingido')
            break

        i += 1 # Próxima iteração
        toc2 = datetime.now() # final processamento da iteração
        time_process = toc2 - tic # tempo total de processamento da iteração
        print(f'  -> Tempo de processamento total: {time_process}', end = '\n\n')
    
    return summary_rfe

In [ ]:
# 10.1.2. Aplicando função para retirada de vars. recursiva
summary = recursive_selection(
    features = feats4,
    X_train = X_des,
    X_test = X_val,
    y_train = y_des,
    y_test = y_val,
    step = 1,          # De quantas em quantas variáveis faremos a recursão? (1 em 1)
    max_ks_diff = 0.2, # Qual diferença máxima de KS estamos dispostos a aceitar? (1 ponto, ou seja estou disposto a perder um 1 de ks desde que meu modelo fique com menos variáveis - CUIDADO)
    n_feats = 5        # Qual número de features final queremos?
)

In [ ]:
# Dataframe sumarizando o processo de RFE
summary

In [ ]:
feats5 = summary['feature'].values[2]

# Verificando a iteração onde o processo determinou o melhor modelo
#feats5 = summary['feature'].values[-1]
#len(feats)
feats5

In [ ]:
# 10.1.3. Retreino do novo modelo com as features restantes
lgb = LGBMClassifier(subsample = 0.7746298211333279, reg_lambda = 0.04676440937077718, reg_alpha = 5.937868749792548, num_leaves = 3, n_estimators = 100, min_split_gain = 16, min_child_weight = 378.8502727624012, min_child_samples = 1900, max_depth = 4, learning_rate = 0.012167884278142268, gamma = 4, colsample_bytree = 0.8332620063985445,
random_state = 2022, # semente aleatória
n_jobs = 10, # quantidade de processadores
importance_type = 'gain'
)
lgb.fit(X_des[feats5], y_des) # Fit do modelo com o X e y de treino

# 10.1.4. Aplicando o modelo (probabilidade) nas amostras. (Lembrar que mau é 1)
p_train_lgb = lgb.predict_proba(X_des[feats5])[:, 1]  # probabilidade do treino
p_test_lgb = lgb.predict_proba(X_val[feats5])[:, 1]    # probabilidade do teste
p_oot_lgb = lgb.predict_proba(X_oot[feats5])[:, 1]      # probabilidade da OOT

# 10.1.5. Print do KS em cada uma das amostras
print('--> LightGBM')
print(f'KS TRAIN: {ks(y_des, p_train_lgb)}')
print(f'KS TEST:  {ks(y_val, p_test_lgb)}')
print(f'KS OOT:   {ks(y_oot, p_oot_lgb)}')

In [ ]:
# 10.1.6. Criar um DataFrame com as importâncias referenciando as features
df_imp = pd.DataFrame({
  'feature': feats5,
  'importance': lgb.feature_importances_
})

# 10.1.7. Ordenando da mais importante para a menos
df_imp.sort_values('importance', ascending = False, inplace=True)

# 10.1.8. Print 5 features mais importantes
df_imp['Perc'] = df_imp['importance']/sum(df_imp['importance'])
df_imp

In [ ]:
feats5

# 11. SHAP

In [ ]:
X__des = X_des[feats5].copy(); X__val = X_val[feats5].copy(); X__oot = X_oot[feats5].copy()

# 11.1. Construção do LGBM
lgb = LGBMClassifier(subsample = 0.7746298211333279, reg_lambda = 0.04676440937077718, reg_alpha = 5.937868749792548, num_leaves = 3, n_estimators = 100, min_split_gain = 16, min_child_weight = 378.8502727624012, min_child_samples = 1900, max_depth = 4, learning_rate = 0.012167884278142268, gamma = 4, colsample_bytree = 0.8332620063985445,
random_state = 2022, # semente aleatória
n_jobs = 10, # quantidade de processadores
importance_type = 'gain'
)
lgb.fit(X__des, y_des) # Fit do modelo com o X e y de treino

# 11.2. Aplicando o modelo (probabilidade) nas amostras.
p_train_lgb = lgb.predict_proba(X__des)[:, 1] # probabilidade do treino
p_test_lgb = lgb.predict_proba(X__val)[:, 1]  # probabilidade do teste
p_oot_lgb = lgb.predict_proba(X__oot)[:, 1]   # probabilidade da OOT

# 11.3. Print do KS em cada uma das amostras
print('--> LightGBM')
print(f'KS TRAIN: {ks(y_des, p_train_lgb)}')
print(f'KS TEST:  {ks(y_val, p_test_lgb)}')
print(f'KS OOT:   {ks(y_oot, p_oot_lgb)}')

In [ ]:
# Summary plot: Gráfico com a distribuição do (impacto) SHAP values (eixo x) para cada feature (eixo y) da mais importânte para a menos importante. 
# As cores representam a amplitude do valor da feature. Quanto mais próximo do verde maior é o valor, e quanto mais próximo do azul menor é o valor.

tree_explainer = shap.TreeExplainer(lgb) # Conversão do modelo para uma árvore explicativa do SHAP (tree explainer)
shap_values_tree = tree_explainer.shap_values(X__des) # Estimativas do shap values através do tree explainer
shap_obj_tree = tree_explainer(X__des) # Aplicando na matriz especificada (X_shap)

shap.summary_plot(
    shap_values = shap_values_tree[1], # Valores do SHAP visando o target 1 (quanto maior o shap value maiores as chances de 1)
    features = X__des, # DataFrame para extrair informações das features
    plot_type = 'dot', # Tipo de plot (pontos)
    cmap = 'vlag', # Paleta de cores (azul-verde)
    max_display = 30, # Máximo de features para mostrar no gráfico (mais importântes),
    plot_size=[8,6],
    show=False
)

import matplotlib.pyplot as plt
plt.savefig('SHAP.png', format='png', dpi=600, bbox_inches='tight')
plt.show()

# 12. Escorando BASE

In [ ]:
# 12.1. Construção do LGBM
lgb = LGBMClassifier(subsample = 0.7746298211333279, reg_lambda = 0.04676440937077718, reg_alpha = 5.937868749792548, num_leaves = 3, n_estimators = 100, min_split_gain = 16, min_child_weight = 378.8502727624012, min_child_samples = 1900, max_depth = 4, learning_rate = 0.012167884278142268, gamma = 4, colsample_bytree = 0.8332620063985445,
random_state = 2022, # semente aleatória
n_jobs = 10, # quantidade de processadores
importance_type = 'gain'
)
lgb.fit(X_des[feats5], y_des) # Fit do modelo com o X e y de treino

# Concat
des = pd.concat([X_des, y_des], axis=1)
val = pd.concat([X_val, y_val], axis=1)
oot = pd.concat([X_oot, y_oot], axis=1)

# Calcula P1 (vetor de probabilidades de ser mau)
des['P1'] = lgb.predict_proba(des[feats5])[:, 1] # probabilidade do treino
val['P1'] = lgb.predict_proba(val[feats5])[:, 1] # probabilidade do teste
oot['P1'] = lgb.predict_proba(oot[feats5])[:, 1] # probabilidade da OOT

# Print do KS em cada uma das amostras
print('--> LightGBM')
print(f"KS TRAIN: {ks(des['RESPOSTA'], des['P1'])}")
print(f"KS TEST:  {ks(val['RESPOSTA'], val['P1'])}")
print(f"KS OOT:   {ks(oot['RESPOSTA'], oot['P1'])}")

# Empilhando
df_scored = pd.concat([des, val, oot], ignore_index=True, axis=0)
df_scored = df_scored.sort_index(ascending=True)

#### 12.1. Cria decil de P1

In [ ]:
##https://stackoverflow.com/questions/20158597/how-to-qcut-with-non-unique-bin-edges
df_scored['rank'] = df_scored['P1'].rank(method='first')
df_scored['rnk_p1'] = pd.qcut(df_scored['rank'].values, 10).codes

##### 12.2. Maus por decil

In [ ]:
# Inadimplência por Safra
df2 = df_scored.groupby('rnk_p1')['RESPOSTA'].agg(['sum', 'count']).rename(columns={'sum':'Maus', 'count':'Total'})             # Agrupa conceito por safra
df2.reset_index()
df2['Bad_Rate'] = 100*(df2['Maus']/df2['Total'])
df2['% Distr'] = df2['Total']/sum(df2['Total'])*100
df2 = df2.reset_index();
df2